# Manual Recipe Modeling Challenges

Setting up relative imports:

In [1]:
import sys, os
sys.path.append(os.path.abspath("../src/"))


## Challenge #1: Partial Transfer and Residues

*Fry bacon until crisp, then remove to a plate. Pour off all but 1 tablespoon of fat, and sauté onions in the same pan.*

Let's start by instantiating our canonical entities:

In [2]:
from dsl.lexicon import CanonicalIngredient, CanonicalTechnique, CanonicalTool, CanonicalLocation, CanonicalContainer, ParamSpec
from dsl.units import StandardUnit


bacon = CanonicalIngredient(name="bacon")
onion = CanonicalIngredient(name="onion")
fat   = CanonicalIngredient(name="bacon fat")

fry = CanonicalTechnique(
    name="fry",
    static=(
        "dry-heat",
        "fat-mediated",
        "pan-contact",
        "shallow-fat",
        "maillard",
    ),
    parameters={
        "fat_type":      ParamSpec(default="neutral-oil"),
        "fat_volume":    ParamSpec(default=0.03, unit=StandardUnit.VOLUME, bounds=(0.005, 0.1)),
        "agit_interval": ParamSpec(default=30.0, unit=StandardUnit.TIME, bounds=(5.0, 60.0)),
    },
)

saute = CanonicalTechnique(
    name="saute",
    static=(
        "dry-heat",
        "fat-mediated",
        "pan-contact",
        "thin-coat",
        "high-heat",
        "agitation",
    ),
    parameters={
        "fat_type":      ParamSpec(default="rendered-fat-or-oil"),
        "fat_volume":    ParamSpec(default=0.015, unit=StandardUnit.VOLUME, bounds=(0.005, 0.03)),
        "agit_interval": ParamSpec(default=5.0, unit=StandardUnit.TIME, bounds=(2.0, 15.0)),
    },
)

stovetop = CanonicalLocation(name="stovetop")
pan      = CanonicalContainer(name="skillet", material="steel", volume=2.0)
plate    = CanonicalContainer(name="plate", material="ceramic")

And then we implement the steps of the recipe itself:

In [3]:
from dsl.ingredient import Ingredient
from dsl.environment import Environment
from dsl.temperature import StaticTemperature
from dsl.action import Process, Transfer, Plate


# step 1: transfer bacon to pan
bacon_in_pan = Transfer(
	inputs     =(Ingredient(ingredient=bacon._uid, quantity=0.25, unit=StandardUnit.WEIGHT, modifiers=("strips",)),),
	destination=Environment(location=stovetop._uid, container=pan._uid)
)

# step 2: fry bacon until crisp
fried_bacon = Process(
	inputs     =(bacon_in_pan._uid,),
	technique  =fry._uid,
	temperature=StaticTemperature(value=190.0),
	condition  ="crisp"
)

# step 3: remove bacon to a plate
plated_bacon = Transfer(
	inputs     =(fried_bacon._uid,),
	destination=Environment(container=plate._uid),
	modifiers  ="pour off all but 1 tbsp fat from pan"
)

# step 4: transfer onions to pan
onion_in_greased_pan = Transfer(
	inputs     =(Ingredient(ingredient=onion._uid, quantity=0.15, unit=StandardUnit.WEIGHT, form="diced"),), # assume available in diced form for this example
	destination=Environment(location=stovetop._uid, container=pan._uid)
)

# step 5: saute onions
sauteed_onions   = Process(
	inputs     =(onion_in_greased_pan._uid,),
	technique  =saute._uid,
	temperature=StaticTemperature(value=160.0),
	condition  ="golden-brown",
	modifiers  ="use remaining bacon fat in same pan"
)

# final recipe
recipe_1 = (bacon_in_pan, fried_bacon, plated_bacon, onion_in_greased_pan, sauteed_onions)

Finally, a quick summary:

In [4]:
for i, step in enumerate(recipe_1, start=1):
    print(f"Step {i}: {step}")

Step 1: Transfer(_uid='kbKJoJDHKsNW', inputs=(Ingredient(ingredient='fOMV91mTbh9R', quantity=0.25, unit=<StandardUnit.WEIGHT: 'kg'>, form=None, modifiers=('strips',)),), destination=Environment(location='27-nv8aANZE4', container='FRXvYZ3gUivp', modifiers=()), time=None, modifiers=None)
Step 2: Process(_uid='t8MC6QZd0H8S', inputs=('kbKJoJDHKsNW',), technique='9xScSgP0QdWL', tool=None, temperature=StaticTemperature(unit=<StandardUnit.TEMPERATURE: 'C'>, value=190.0), time=None, condition='crisp', modifiers=None)
Step 3: Transfer(_uid='E-ZCQIJ-JAzG', inputs=('t8MC6QZd0H8S',), destination=Environment(location=None, container='gk468B7NVlHG', modifiers=()), time=None, modifiers='pour off all but 1 tbsp fat from pan')
Step 4: Transfer(_uid='OatpyAAF65GX', inputs=(Ingredient(ingredient='xHMQfxlsLTFu', quantity=0.15, unit=<StandardUnit.WEIGHT: 'kg'>, form='diced', modifiers=()),), destination=Environment(location='27-nv8aANZE4', container='FRXvYZ3gUivp', modifiers=()), time=None, modifiers=None)

## Challenge #2: Concurrent Interjections, Ramped Heat, and Easing Curves

*Poach fish in oil at about 50°C, gradually increase heat to about 80°C over 18 minutes (speed up the increase as you go). Baste every 30 seconds and add a tablespoon of butter at 6, 12, and 15 minutes.*

Starting again with canonical entities:

In [5]:
from dsl.lexicon import CanonicalIngredient, CanonicalTechnique, CanonicalLocation, CanonicalContainer, ParamSpec
from dsl.units import StandardUnit


fish   = CanonicalIngredient("halibut")
oil    = CanonicalIngredient("olive oil")
butter = CanonicalIngredient("butter")

baste = CanonicalTechnique(
    name="baste",
    static=(
        "fat-mediated",
        "surface-application",
        "spoon-baste",
        "intermittent",
    ),
    parameters={
        # "interval":    ParamSpec(default=30.0, unit=StandardUnit.TIME, bounds=(5.0, 120.0)),
        "volume":      ParamSpec(default=0.015, unit=StandardUnit.VOLUME, bounds=(0.002, 0.05)),
    },
)

poach = CanonicalTechnique(
    name="poach",
    static=(
        "moist-heat",
        "fat-mediated",
        "gentle",
        "submersion",
    ),
    parameters={
        "fat_type":    ParamSpec(default="neutral-oil"),
    },
)

stovetop = CanonicalLocation(name="stovetop")
pot      = CanonicalContainer(name="saucepan", material="steel", volume=3.0)

And now the recipe itself:

In [6]:
from dsl.temperature import RampTemperature
from dsl.timing import Timing
from dsl.curves import exp_increase


# step 1: set up oil bath with fish
fish_in_saucepan = Transfer(
    inputs=(
        Ingredient(ingredient=fish._uid, quantity=0.18, unit=StandardUnit.WEIGHT),
        Ingredient(ingredient=oil._uid,  quantity=0.8,  unit=StandardUnit.VOLUME),
    ),
    destination=Environment(location=stovetop._uid, container=pot._uid),
)

# step 2.0: poach along an exponential (accelerating) ramp from 55C to 82C over 18 min
poached_fish = Process(
    inputs     =(fish_in_saucepan._uid,),
    technique  =poach._uid,
    temperature=RampTemperature(start=55.0, end=82.0, curve=exp_increase),
    time       =Timing(value=18.0 * 60.0),
)

# step 2.1: concurrent basting interjection (every 30s, non-blocking, repeating)
poached_fish__baste_loop = Process(
    inputs   =(fish_in_saucepan._uid,),
    technique=baste._uid,
    time     =Timing(value=30.0, relative_to=poached_fish._uid, repeating=True),
)

# step 2.2: fractional butter additions at 6, 12, and 15 minutes (non-blocking)
poached_fish__add_butter = Transfer(
    inputs     =(Ingredient(ingredient=butter._uid, quantity=0.025, unit=StandardUnit.WEIGHT),),
    destination=fish_in_saucepan._uid,
    time       =Timing(value=0.333, relative_to=poached_fish._uid, repeating=True),
)

# final recipe
recipe_2 = (fish_in_saucepan, poached_fish, poached_fish__baste_loop, poached_fish__add_butter)

Final summary:

In [7]:
for i, step in enumerate(recipe_2, start=1):
    print(f"Step {i}: {step}")

Step 1: Transfer(_uid='xwYtMETmVkcr', inputs=(Ingredient(ingredient='YUnmOQ2O2S2S', quantity=0.18, unit=<StandardUnit.WEIGHT: 'kg'>, form=None, modifiers=()), Ingredient(ingredient='5PtQEF4_rOrO', quantity=0.8, unit=<StandardUnit.VOLUME: 'l'>, form=None, modifiers=())), destination=Environment(location='0RHmT9cLrBZ-', container='humWYNKv1Ile', modifiers=()), time=None, modifiers=None)
Step 2: Process(_uid='X8Y4ZTBxj-6H', inputs=('xwYtMETmVkcr',), technique='JXfGpH39bAYK', tool=None, temperature=RampTemperature(unit=<StandardUnit.TEMPERATURE: 'C'>, start=55.0, end=82.0, curve=<function exp_increase at 0x7fbc5eb2e0c0>), time=Timing(unit=<StandardUnit.TIME: 's'>, value=1080.0, relative_to=None, blocking=False, repeating=False), condition=None, modifiers=None)
Step 3: Process(_uid='e16yyioIg1MK', inputs=('xwYtMETmVkcr',), technique='keXd0jAfa6T5', tool=None, temperature=None, time=Timing(unit=<StandardUnit.TIME: 's'>, value=30.0, relative_to='X8Y4ZTBxj-6H', blocking=False, repeating=True),

## Challenge #3: Iterative Addition, Doneness-by-Feel, Off-Heat Emulsion

*Make classic cacio e pepe: cook spaghetti until 1 minute shy of al dente; transfer to a wide pan with a splash of starchy water and cracked pepper, tossing vigorously and adding water as needed to maintain a glossy suspension. Off heat, rain in finely grated Pecorino in 2–3 additions while tossing, keeping the sauce below ~65°C to avoid clumping; adjust with pasta water and seasoning "to taste" until the sauce is glossy and flows.*

**[LLM Soln.]** Starting again with canonical entities:

In [8]:
from dsl.lexicon import CanonicalIngredient, CanonicalTechnique, CanonicalContainer, ParamSpec
from dsl.units import StandardUnit

# canonical ingredients (challenge #3)
spaghetti  = CanonicalIngredient("spaghetti")
pecorino   = CanonicalIngredient("pecorino romano")
pepper     = CanonicalIngredient("black pepper")
pasta_water= CanonicalIngredient("pasta water")

# techniques
boil = CanonicalTechnique(
    name="boil",
    static=("moist-heat", "water-mediated", "submersion"),
    parameters={
        "salt_pct": ParamSpec(default=0.02),
    },
)

toss = CanonicalTechnique(
    name="toss",
    static=("agitation", "pan-toss"),
    parameters={}
)

emulsify = CanonicalTechnique(
    name="emulsify",
    static=("emulsion", "off-heat"),
    parameters={
        "temperature": ParamSpec(default=60.0, unit=StandardUnit.TEMPERATURE, bounds=(30.0, 65.0)),
    },
)

# containers
wide_pan = CanonicalContainer(name="wide pan", material="steel", volume=3.0)

And now the recipe itself:

In [9]:
from dsl.ingredient import Ingredient
from dsl.environment import Environment
from dsl.temperature import StaticTemperature
from dsl.action import Process, Transfer
from dsl.timing import Timing
from dsl.units import StandardUnit

# step 1: cook spaghetti until 1 minute shy of al dente
spaghetti_in_pot = Transfer(
    inputs=(Ingredient(ingredient=spaghetti._uid, quantity=0.24, unit=StandardUnit.WEIGHT),),
    destination=Environment(location=stovetop._uid, container=pot._uid),
    modifiers="in generously salted boiling water",
)
pasta_near_al_dente = Process(
    inputs=(spaghetti_in_pot._uid,),
    technique=boil._uid,
    condition="1-min-shy-of-al-dente",
)

# step 2: transfer to wide pan with splash of starchy water + pepper
pasta_in_wide_pan = Transfer(
    inputs=(
        pasta_near_al_dente._uid,
        Ingredient(ingredient=pasta_water._uid, quantity=0.1, unit=StandardUnit.VOLUME),
        Ingredient(ingredient=pepper._uid,      quantity=0.002, unit=StandardUnit.WEIGHT, form="cracked"),
    ),
    destination=Environment(container=wide_pan._uid),
)

# step 3: vigorous tossing with on-demand water splashes to maintain a glossy suspension
tossing = Process(
    inputs=(pasta_in_wide_pan._uid,),
    technique=toss._uid,
    time=Timing(value=10.0, repeating=True),
    modifiers="vigorous tossing; add splashes as needed",
)

# step 4: off-heat cheese emulsification in two additions, keep below ~65C
cheese_add_1 = Process(
    inputs=(tossing._uid, Ingredient(ingredient=pecorino._uid, quantity=0.06, unit=StandardUnit.WEIGHT)),
    technique=emulsify._uid,
    temperature=StaticTemperature(value=60.0),
    time=Timing(value=5.0, relative_to=tossing._uid, blocking=False),
    modifiers="first rain of cheese, off heat",
)
cheese_add_2 = Process(
    inputs=(cheese_add_1._uid, Ingredient(ingredient=pecorino._uid, quantity=0.04, unit=StandardUnit.WEIGHT)),
    technique=emulsify._uid,
    temperature=StaticTemperature(value=60.0),
    time=Timing(value=10.0, relative_to=cheese_add_1._uid, blocking=False),
    modifiers="second rain of cheese, off heat",
)

# step 5: adjust with pasta water and seasoning to taste until sauce is glossy and flows
finish = Process(
    inputs=(cheese_add_2._uid, Ingredient(ingredient=pasta_water._uid, quantity=0.02, unit=StandardUnit.VOLUME)),
    technique=toss._uid,
    condition="glossy-and-flowing; to-taste",
)

recipe_3 = (spaghetti_in_pot, pasta_near_al_dente, pasta_in_wide_pan, tossing, cheese_add_1, cheese_add_2, finish)

Final summary:

In [10]:
for i, step in enumerate(recipe_3, start=1):
    print(f"Step {i}: {step}")

Step 1: Transfer(_uid='0FAGtGDVfCWl', inputs=(Ingredient(ingredient='Ftl51w_kvdfF', quantity=0.24, unit=<StandardUnit.WEIGHT: 'kg'>, form=None, modifiers=()),), destination=Environment(location='0RHmT9cLrBZ-', container='humWYNKv1Ile', modifiers=()), time=None, modifiers='in generously salted boiling water')
Step 2: Process(_uid='Ck-j9xvs5cTZ', inputs=('0FAGtGDVfCWl',), technique='wcfnepyOyJ_g', tool=None, temperature=None, time=None, condition='1-min-shy-of-al-dente', modifiers=None)
Step 3: Transfer(_uid='ndAjJ0_LwbJk', inputs=('Ck-j9xvs5cTZ', Ingredient(ingredient='aavhiWYzyaIN', quantity=0.1, unit=<StandardUnit.VOLUME: 'l'>, form=None, modifiers=()), Ingredient(ingredient='hAB8o3XZ4nSG', quantity=0.002, unit=<StandardUnit.WEIGHT: 'kg'>, form='cracked', modifiers=())), destination=Environment(location=None, container='zLZWw6jUujO-', modifiers=()), time=None, modifiers=None)
Step 4: Process(_uid='8Y01MaaCBp8q', inputs=('ndAjJ0_LwbJk',), technique='g6s83yYnEElk', tool=None, temperatur